# Data Processsing

In [1]:
train_dir = '/home/ec2-user/SageMaker/train/'
test_dir = '/home/ec2-user/SageMaker/test/'
files_dir = '/home/ec2-user/SageMaker/'

## Labels

In [2]:
import json

with open(files_dir + '/label_num_to_disease_map.json', 'r') as f:
    labels_info = json.load(f)

NUM_LABELS = len(labels_info)
print(labels_info)

{'0': 'Cassava Bacterial Blight (CBB)', '1': 'Cassava Brown Streak Disease (CBSD)', '2': 'Cassava Green Mottle (CGM)', '3': 'Cassava Mosaic Disease (CMD)', '4': 'Healthy'}


## Load Data with Pytorch

In [3]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("We're using:", device)

We're using: cuda


In [5]:
df = pd.read_csv(files_dir + '/train.csv')

# Split the data into train, validation, and test sets with a ratio of 80:10:10
train_df, val_test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=0.5, random_state=42)

## regular images (non-transformed)

In [6]:
class CassavaDataset(Dataset):
    def __init__(self, data, root_dir, transform=None):
        self.image_labels = data
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.image_labels.iloc[index, 0])
        image = Image.open(img_path).convert('RGB')
        label = self.image_labels.iloc[index, 1]
        if self.transform:
            image = self.transform(image)
        return image, label

# Useful Functions

In [7]:
# Save the model so that we don't need to train the same model multiple times
def save_model(model, model_name):
  torch.save(model.state_dict(),  files_dir + 'saved_models/' + model_name + '.pth')

# Vision Transformer

### Model Parameters

In [8]:
BATCH_SIZE = 16
EPOCHS = 10
LR = 2e-05
IMG_SIZE = 224

### Transforms

In [9]:
# For training -- includes two augmentations
transforms_train = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(p=0.3),
        transforms.RandomVerticalFlip(p=0.3),
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transforms_valid = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [10]:
train_dataset = CassavaDataset(train_df, train_dir, transform=transforms_train)
validation_dataset = CassavaDataset(val_df, train_dir, transform=transforms_valid)
test_dataset = CassavaDataset(test_df, train_dir, transform=transforms_valid)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### Transformer Model

In [12]:
!pip install timm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [13]:
import timm

In [14]:
model = timm.create_model("vit_base_patch16_224", pretrained=False)
# model.head = torch.nn.Linear(model.head.in_features, NUM_LABELS)
model.head = torch.nn.Sequential(
    torch.nn.Linear(model.head.in_features, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, NUM_LABELS)
)


In [15]:
model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

### Train Model

In [16]:
train_loss, validation_loss = [], []
train_acc, validation_acc = [], []

for epoch in range(EPOCHS):
    print("Training Epoch: ", epoch)
    batch_num = 0
    
    model.train()
    running_loss = 0.
    correct, total = 0, 0 
    total_acc= 0.0
    for inputs, labels in train_loader:
        batch_num += 1
        if batch_num % 50 == 0:
            print("Current Batch: ", batch_num)
        # Forward pass
        inputs = inputs.to(device)
        labels = labels.to(device)
         
        predictions = model(inputs)
        loss = criterion(predictions, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        acc = (predictions.argmax(dim=1) == labels).float().mean()
        total_acc += acc
        
        running_loss += loss.item()
            
        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    train_loss.append(running_loss / len(train_loader))
    train_acc.append(correct/total)
    print("Train Accuracy:", total_acc / total)

    model.eval()
    running_loss = 0.
    correct, total = 0, 0 
    
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        predictions = model(inputs)

        loss = criterion(predictions, labels)

        running_loss += loss.item()

        _, predicted = torch.max(predictions, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    validation_loss.append(running_loss / len(val_loader))
    validation_acc.append(correct/total)
    
    print(f"Epoch {epoch+1}:")

    print(f"Training Loss:", round(train_loss[epoch], 3))
    print(f"Validation Loss:", round(validation_loss[epoch], 3))

    print(f"Training Accuracy:", round(train_acc[epoch], 3))
    print(f"Validation Accuracy:", round(validation_acc[epoch], 3))

    print("------------------------------")

Training Epoch:  0
Current Batch:  50
Current Batch:  100
Current Batch:  150
Current Batch:  200
Current Batch:  250
Current Batch:  300
Current Batch:  350
Current Batch:  400
Current Batch:  450
Current Batch:  500
Current Batch:  550
Current Batch:  600
Current Batch:  650
Current Batch:  700
Current Batch:  750
Current Batch:  800
Current Batch:  850
Current Batch:  900
Current Batch:  950
Current Batch:  1000
Current Batch:  1050
Train Accuracy: tensor(0.0390, device='cuda:0')
Epoch 1:
Training Loss: 1.074
Validation Loss: 1.008
Training Accuracy: 0.623
Validation Accuracy: 0.647
------------------------------
Training Epoch:  1
Current Batch:  50
Current Batch:  100
Current Batch:  150
Current Batch:  200
Current Batch:  250
Current Batch:  300
Current Batch:  350
Current Batch:  400
Current Batch:  450
Current Batch:  500
Current Batch:  550
Current Batch:  600
Current Batch:  650
Current Batch:  700
Current Batch:  750
Current Batch:  800
Current Batch:  850
Current Batch:  90

In [17]:
save_model(model, "vit_10_epochs_16_batch")

### Model Evaluation

In [18]:
model.eval()

test_predictions = np.array([])
true_labels = np.array([])

total = 0
correct = 0

for inputs, labels in test_loader:
    
    inputs = inputs.to(device)
    labels = labels.to(device)

    predictions = model(inputs)
    
    _, predicted = torch.max(predictions, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
    
    true_labels = np.append(true_labels, labels.cpu().numpy())
    test_predictions = np.concatenate((test_predictions, predicted.detach().cpu().numpy()))
    
print("Test Accuracy: ", correct/total)

Test Accuracy:  0.7224299065420561


In [19]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

f1 = f1_score(true_labels, test_predictions, average='weighted')
recall = recall_score(true_labels, test_predictions, average='weighted')
precision = precision_score(true_labels, test_predictions, average='weighted')
accuracy = accuracy_score(true_labels, test_predictions)

# Print the accuracy, F1 score, recall, and precision
print("Test Accuracy: ", accuracy)
print("F1 Score: ", f1)
print("Recall: ", recall)
print("Precision: ", precision)

Test Accuracy:  0.7224299065420561
F1 Score:  0.6856029520564499
Recall:  0.7224299065420561
Precision:  0.6902457320080138
